In [ ]:
!pip install quantities

In [ ]:
!rm simoc_server/db.sqlite

In [ ]:
import json

In [ ]:
import pprint
pp = pprint.PrettyPrinter()

In [ ]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

In [ ]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

In [ ]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

In [ ]:
AGENTS = {
    "greenhouse_medium": 1,
    "human_agent": 1,
    "cabbage": 10,
    'lettuce': 10,
    'solar_pv_array': 10,
    'multifiltration_purifier_post_treament': 10
}

STORAGES = {
    "air_storage": 2,
    "water_storage": 1,
    "nutrient_storage": 1,
    "power_storage": 1,
    "food_storage": 1
}

game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(AGENTS, STORAGES)
game_runner_manager.new_game(user, game_runner_init_params)

In [ ]:
num_steps = 86 * 24
for i in range(1, num_steps + 1):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)